In [24]:
import polars as pl
import polars.selectors as cs
import os

In [28]:
volunteers = pl.scan_parquet("dataset/gz_decals_volunteers_5.parquet")
volunteers.head().collect()

iauname,ra,dec,redshift,elpetro_absmag_r,sersic_nmgy_r,petro_th50,petro_th90,petro_theta,upload_group,active_learning_on,in_gzd_ab,png_loc,smooth-or-featured_total-votes,smooth-or-featured_smooth,smooth-or-featured_smooth_fraction,smooth-or-featured_smooth_debiased,smooth-or-featured_featured-or-disk,smooth-or-featured_featured-or-disk_fraction,smooth-or-featured_featured-or-disk_debiased,smooth-or-featured_artifact,smooth-or-featured_artifact_fraction,smooth-or-featured_artifact_debiased,disk-edge-on_total-votes,disk-edge-on_yes,disk-edge-on_yes_fraction,disk-edge-on_yes_debiased,disk-edge-on_no,disk-edge-on_no_fraction,disk-edge-on_no_debiased,has-spiral-arms_total-votes,has-spiral-arms_yes,has-spiral-arms_yes_fraction,has-spiral-arms_yes_debiased,has-spiral-arms_no,has-spiral-arms_no_fraction,has-spiral-arms_no_debiased,…,spiral-winding_loose,spiral-winding_loose_fraction,spiral-winding_loose_debiased,spiral-arm-count_total-votes,spiral-arm-count_1,spiral-arm-count_1_fraction,spiral-arm-count_1_debiased,spiral-arm-count_2,spiral-arm-count_2_fraction,spiral-arm-count_2_debiased,spiral-arm-count_3,spiral-arm-count_3_fraction,spiral-arm-count_3_debiased,spiral-arm-count_4,spiral-arm-count_4_fraction,spiral-arm-count_4_debiased,spiral-arm-count_more-than-4,spiral-arm-count_more-than-4_fraction,spiral-arm-count_more-than-4_debiased,spiral-arm-count_cant-tell,spiral-arm-count_cant-tell_fraction,spiral-arm-count_cant-tell_debiased,merging_total-votes,merging_none,merging_none_fraction,merging_none_debiased,merging_minor-disturbance,merging_minor-disturbance_fraction,merging_minor-disturbance_debiased,merging_major-disturbance,merging_major-disturbance_fraction,merging_major-disturbance_debiased,merging_merger,merging_merger_fraction,merging_merger_debiased,wrong_size_statistic,wrong_size_warning
str,f64,f64,f32,f32,f32,f32,f32,f32,str,bool,bool,str,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,…,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,f64,bool
"""J112953.88-000427.4""",172.474526,-0.074281,0.124142,-21.253342,167.262878,1.835066,5.414066,4.202472,"""pre_active""",false,true,"""dr5/J112/J112953.88-000427.4.p…",84,57,0.678571,0.102564,23,0.27381,0.916667,4,0.047619,0.025742,23,7,0.304348,0.04878,16,0.695652,0.805502,16,1,0.0625,0.820513,15,0.9375,0.108171,…,0,0.0,0.0,1,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,1,1.0,1.0,80,67,0.8375,0.772727,8,0.1,0.127708,5,0.0625,0.076923,0,0.0,0.0,133.592684,false
"""J104325.29+190335.0""",160.856533,19.060441,0.049088,-21.77541,2941.729248,4.405412,14.320828,10.757237,"""targeted""",false,false,"""dr5/J104/J104325.29+190335.0.p…",37,33,0.891892,0.857143,2,0.054054,0.038462,2,0.054054,0.022166,2,0,0.0,0.0,2,1.0,1.0,2,0,0.0,0.0,2,1.0,1.0,…,0,null,0.0,0,0,null,0.0,0,null,0.0,0,null,0.0,0,null,0.0,0,null,0.0,0,null,0.0,35,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,2,0.057143,0.060757,132.443129,false
"""J104629.54+115415.1""",161.623135,11.904197,0.092906,-19.947397,125.076324,4.71987,13.139816,9.726173,"""active_baseline""",false,false,"""dr5/J104/J104629.54+115415.1.p…",5,1,0.2,null,4,0.8,null,0,0.0,null,4,0,0.0,null,4,1.0,null,4,4,1.0,null,0,0.0,null,…,1,0.25,null,4,0,0.0,null,4,1.0,null,0,0.0,null,0,0.0,null,0,0.0,null,0,0.0,null,5,4,0.8,null,1,0.2,null,0,0.0,null,0,0.0,null,156.336302,false
"""J082950.68+125621.8""",127.46119,12.939386,0.066619,-19.800629,173.592026,2.929065,7.224961,6.528331,"""active_baseline""",false,false,"""dr5/J082/J082950.68+125621.8.p…",8,2,0.25,null,6,0.75,null,0,0.0,null,6,6,1.0,null,0,0.0,null,0,0,null,null,0,null,null,…,0,null,null,0,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,8,7,0.875,null,1,0.125,null,0,0.0,null,0,0.0,null,155.386379,false
"""J122056.00-015022.0""",185.233342,-1.83934,0.070042,-19.013256,88.826881,2.521897,6.496888,5.054409,"""active_baseline""",false,false,"""dr5/J122/J122056

In [29]:
def add_image_path(df: pl.LazyFrame, dataset_dir: str = "./dataset") -> pl.LazyFrame:
    # check if file exists
    def file_exists(path: str) -> bool:
        return os.path.exists(os.path.join(dataset_dir, path))

    # drop first component of "png_loc"
    df = (
        df.with_columns(pl.col("png_loc").str.strip_prefix("dr5/").alias("image_path"))
        .filter(pl.col("image_path").map_elements(file_exists, return_dtype=pl.Boolean))
        .drop("png_loc")
    )

    # reorder columns to have 'image_path' first
    df = df.select(
        pl.col("iauname", "image_path"),
        *df.drop("image_path", "iauname").collect_schema().names(),
    )

    return df


volunteers_with_images = add_image_path(volunteers)
volunteers_with_images.tail().collect()

iauname,image_path,ra,dec,redshift,elpetro_absmag_r,sersic_nmgy_r,petro_th50,petro_th90,petro_theta,upload_group,active_learning_on,in_gzd_ab,smooth-or-featured_total-votes,smooth-or-featured_smooth,smooth-or-featured_smooth_fraction,smooth-or-featured_smooth_debiased,smooth-or-featured_featured-or-disk,smooth-or-featured_featured-or-disk_fraction,smooth-or-featured_featured-or-disk_debiased,smooth-or-featured_artifact,smooth-or-featured_artifact_fraction,smooth-or-featured_artifact_debiased,disk-edge-on_total-votes,disk-edge-on_yes,disk-edge-on_yes_fraction,disk-edge-on_yes_debiased,disk-edge-on_no,disk-edge-on_no_fraction,disk-edge-on_no_debiased,has-spiral-arms_total-votes,has-spiral-arms_yes,has-spiral-arms_yes_fraction,has-spiral-arms_yes_debiased,has-spiral-arms_no,has-spiral-arms_no_fraction,has-spiral-arms_no_debiased,…,spiral-winding_loose,spiral-winding_loose_fraction,spiral-winding_loose_debiased,spiral-arm-count_total-votes,spiral-arm-count_1,spiral-arm-count_1_fraction,spiral-arm-count_1_debiased,spiral-arm-count_2,spiral-arm-count_2_fraction,spiral-arm-count_2_debiased,spiral-arm-count_3,spiral-arm-count_3_fraction,spiral-arm-count_3_debiased,spiral-arm-count_4,spiral-arm-count_4_fraction,spiral-arm-count_4_debiased,spiral-arm-count_more-than-4,spiral-arm-count_more-than-4_fraction,spiral-arm-count_more-than-4_debiased,spiral-arm-count_cant-tell,spiral-arm-count_cant-tell_fraction,spiral-arm-count_cant-tell_debiased,merging_total-votes,merging_none,merging_none_fraction,merging_none_debiased,merging_minor-disturbance,merging_minor-disturbance_fraction,merging_minor-disturbance_debiased,merging_major-disturbance,merging_major-disturbance_fraction,merging_major-disturbance_debiased,merging_merger,merging_merger_fraction,merging_merger_debiased,wrong_size_statistic,wrong_size_warning
str,str,f64,f64,f32,f32,f32,f32,f32,f32,str,bool,bool,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,…,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,f64,bool
"""J095102.66+284526.4""","""J095/J095102.66+284526.4.png""",147.761108,28.757334,0.115866,-20.813797,133.822235,1.713884,5.054409,3.801523,"""active_baseline""",false,true,5,5,1.0,null,0,0.0,null,0,0.0,null,0,0,null,null,0,null,null,0,0,null,null,0,null,null,…,0,null,null,0,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,5,4,0.8,null,0,0.0,null,1,0.2,null,0,0.0,null,145.215592,false
"""J100725.49+120540.1""","""J100/J100725.49+120540.1.png""",151.852902,12.095359,-0.000057,-8.749258,10117.243164,73.711433,97.296509,105.420822,"""active_baseline""",false,false,5,0,0.0,null,0,0.0,null,5,1.0,null,0,0,null,null,0,null,null,0,0,null,null,0,null,null,…,0,null,null,0,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,0,0,null,null,0,null,null,0,null,null,0,null,null,153.672061,false
"""J091211.32+264824.4""","""J091/J091211.32+264824.4.png""",138.04727,26.806751,0.044191,-18.771126,161.7444,4.18035,12.865334,9.373409,"""active_baseline""",false,true,5,4,0.8,null,0,0.0,null,1,0.2,null,0,0,null,null,0,null,null,0,0,null,null,0,null,null,…,0,null,null,0,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,4,4,1.0,null,0,0.0,null,0,0.0,null,0,0.0,null,154.808065,false
"""J091401.42+344438.1""","""J091/J091401.42+344438.1.png""",138.505922,34.74393,0.117608,-20.988338,183.065918,2.376635,7.086283,5.190236,"""active_baseline""",false,false,5,5,1.0,null,0,0.0,null,0,0.0,null,0,0,null,null,0,null,null,0,0,null,null,0,null,null,…,0,null,null,0,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,5,5,1.0,null,0,0.0,null,0,0.0,null,0,0.0,null,147.941604,false
"""J022627.30-161841.2""","""J022/J022627.30-161841.2.png""",36.613716,-16.311507,0.066226,-22.005053,1883.144409,4.534743,15.287906,10.88915,"""active_baseline""",false,false,4,3,0.75,null,0,0.0,null,1,0.25,null,0,0,null

In [39]:
def make_categories(df: pl.LazyFrame) -> pl.LazyFrame:
    galaxy_enum = pl.Enum(["elliptical", "spiral", "irregular"])

    df = df.with_columns(
        pl.when(pl.col("smooth-or-featured_smooth_fraction") >= 0.5)
        .then(pl.lit("elliptical"))
        .when(pl.col("has-spiral-arms_yes_fraction") >= 0.5)
        .then(pl.lit("spiral"))
        .otherwise(pl.lit("irregular"))
        .cast(galaxy_enum)
        .alias("galaxy_class")
    ).drop_nulls("galaxy_class").with_columns(
        pl.col("galaxy_class").to_physical().cast(pl.Int8).alias("galaxy_class_int"),
    )

    return df

volunteers_with_categories = make_categories(volunteers_with_images)
volunteers_with_categories.head().collect()

iauname,image_path,ra,dec,redshift,elpetro_absmag_r,sersic_nmgy_r,petro_th50,petro_th90,petro_theta,upload_group,active_learning_on,in_gzd_ab,smooth-or-featured_total-votes,smooth-or-featured_smooth,smooth-or-featured_smooth_fraction,smooth-or-featured_smooth_debiased,smooth-or-featured_featured-or-disk,smooth-or-featured_featured-or-disk_fraction,smooth-or-featured_featured-or-disk_debiased,smooth-or-featured_artifact,smooth-or-featured_artifact_fraction,smooth-or-featured_artifact_debiased,disk-edge-on_total-votes,disk-edge-on_yes,disk-edge-on_yes_fraction,disk-edge-on_yes_debiased,disk-edge-on_no,disk-edge-on_no_fraction,disk-edge-on_no_debiased,has-spiral-arms_total-votes,has-spiral-arms_yes,has-spiral-arms_yes_fraction,has-spiral-arms_yes_debiased,has-spiral-arms_no,has-spiral-arms_no_fraction,has-spiral-arms_no_debiased,…,spiral-winding_loose_debiased,spiral-arm-count_total-votes,spiral-arm-count_1,spiral-arm-count_1_fraction,spiral-arm-count_1_debiased,spiral-arm-count_2,spiral-arm-count_2_fraction,spiral-arm-count_2_debiased,spiral-arm-count_3,spiral-arm-count_3_fraction,spiral-arm-count_3_debiased,spiral-arm-count_4,spiral-arm-count_4_fraction,spiral-arm-count_4_debiased,spiral-arm-count_more-than-4,spiral-arm-count_more-than-4_fraction,spiral-arm-count_more-than-4_debiased,spiral-arm-count_cant-tell,spiral-arm-count_cant-tell_fraction,spiral-arm-count_cant-tell_debiased,merging_total-votes,merging_none,merging_none_fraction,merging_none_debiased,merging_minor-disturbance,merging_minor-disturbance_fraction,merging_minor-disturbance_debiased,merging_major-disturbance,merging_major-disturbance_fraction,merging_major-disturbance_debiased,merging_merger,merging_merger_fraction,merging_merger_debiased,wrong_size_statistic,wrong_size_warning,galaxy_class,galaxy_class_int
str,str,f64,f64,f32,f32,f32,f32,f32,f32,str,bool,bool,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,…,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,f64,bool,enum,i8
"""J082950.68+125621.8""","""J082/J082950.68+125621.8.png""",127.46119,12.939386,0.066619,-19.800629,173.592026,2.929065,7.224961,6.528331,"""active_baseline""",false,false,8,2,0.25,null,6,0.75,null,0,0.0,null,6,6,1.0,null,0,0.0,null,0,0,null,null,0,null,null,…,null,0,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,0,null,null,8,7,0.875,null,1,0.125,null,0,0.0,null,0,0.0,null,155.386379,false,"""irregular""",2
"""J100927.56+071112.4""","""J100/J100927.56+071112.4.png""",152.36483,7.186803,0.100948,-21.110653,283.301788,3.722095,9.933681,9.42301,"""pre_active""",false,false,34,14,0.411765,0.05,18,0.529412,0.938776,2,0.058824,0.019949,18,1,0.055556,0.027778,17,0.944444,0.978958,17,7,0.411765,0.907407,10,0.588235,0.310743,…,0.0,7,0,0.0,0.0,4,0.571429,0.225806,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,3,0.428571,0.478261,32,9,0.28125,0.37931,11,0.34375,0.363424,8,0.25,0.378378,4,0.125,0.129736,151.761221,false,"""irregular""",2
"""J095431.99+064902.7""","""J095/J095431.99+064902.7.png""",148.633357,6.817409,0.073774,-21.567976,1003.406433,5.456669,17.837217,12.704952,"""pre_active""",false,false,39,0,0.0,0.0,39,1.0,1.0,0,0.0,0.0,39,1,0.025641,0.026316,38,0.974359,0.987259,38,37,0.973684,0.981818,1,0.026316,0.041504,…,0.268252,37,1,0.027027,0.013333,35,0.945946,0.754717,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,1,0.027027,0.029412,39,28,0.717949,0.769231,6,0.153846,0.168317,4,0.102564,0.118644,1,0.025641,0.028613,142.099513,false,"""spiral""",1
"""J002905.51+020116.8""","""J002/J002905.51+020116.8.png""",7.273156,2.02129,0.078009,-20.600267,305.888702,3.92381,10.472028,9.62405,"""active_priority""",true,false,41,16,0.390244,0.09589,25,0.609756,0.878788,0,0.0,0.0,25,21,0.84,0.561644,4,0.16,0.194238,4,1,0.25,0.873016,3,0.75,0.441618,…,0.0,1,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,1,1.0,1.0,41,28,0.682927,0.648649,10,0.243902,0.262304,2,0.04878,0.026316,1,0.02439,0.027549

In [40]:
volunteers_with_categories.collect().shape

(71883, 129)

In [41]:
# save the updated DataFrame to a new Parquet file
volunteers_with_categories.sink_parquet("dataset/gz_decals_volunteers_5_images.parquet")

In [20]:
import dataset

data = dataset.GalaxyZooDecalsDataset(root="./dataset")

In [21]:
len(data)

18788